In [ ]:
import ctypes #tablice niskopoziomowe

In [ ]:
#zaimplementowanie klasy dynamic array z wykładu

class DynamicArray:

    def __init__(self):
        self._n = 0     #liczba elementów
        self._capacity = 1      #rozmiar tablicy
        self._A = self._make_array(self._capacity)      #właściwa tablica
    
    def __len__(self):
        return self._n
    
    def __getitem__(self, k):
        if not 0 <= k < self._n:
            raise IndexError("invalid index")
        return self._A[k]
    
    def append(self,obj):
        if self._n == self._capacity:
            self._resize(2*self._capacity)
        self._A[self._n] = obj
        self._n += 1

    def resize(self,c):
        B = self._make_array(c)
        for k in range(self._n):
            B[k] = self._A[k]
        self._A = B
        self._capacity = c

    def _make_array(self,c):
        return(c*ctypes.py_object)()